In [28]:
params_big = standalone_lookup(
    tf=21.5,
    tw=12,
    b=300,
    d=340
)
params_small = standalone_lookup(
    tf=8.5,
    tw=5.5,
    b=140,
    d=133
)
tw_small = params_small['tw']
tw_small_dev = abs(tw_small - 5.5) / 5.5
tf_small = params_small['tf']
tf_small_dev = abs(tf_small - 8.5) / 8.5
b_small = params_small['bf']
b_small_dev = abs(b_small - 140) / 140
d_small = params_small['d']
d_small_dev = abs(d_small - 133) / 133

tw_big = params_big['tw']
tw_big_dev = abs(tw_big - 12) / 12
tf_big = params_big['tf']
tf_big_dev = abs(tf_big - 21.5) / 21.5
b_big = params_big['bf']
b_big_dev = abs(b_big - 300) / 300
d_big = params_big['d']
d_big_dev = abs(d_big - 340) / 340

print(f'small beam input: tw=5.5, tf=8.5, b=140, d=133')
print(f'small beam retrieved: tw={tw_small}, tf={tf_small}, b={b_small}, d={d_small}')
print(f'error: tw={tw_small_dev}, tf={tf_small_dev}, b={b_small_dev}, d={d_small_dev}')
print(f'type of small beam: {params_small["type"]}, label: {params_small["label"]}')

print(f'big beam input: tw=12, tf=21.5, b=300, d=340')
print(f'big beam retrieved: tw={tw_big}, tf={tf_big}, b={b_big}, d={d_big}')
print(f'error: tw={tw_big_dev}, tf={tf_big_dev}, b={b_big_dev}, d={d_big_dev}')
print(f'type of big beam: {params_big["type"]}, label: {params_big["label"]}')

small beam input: tw=5.5, tf=8.5, b=140, d=133
small beam retrieved: tw=6.86, tf=10.9, b=128.0, d=131.0
error: tw=0.24727272727272734, tf=0.28235294117647064, b=0.08571428571428572, d=0.015037593984962405
type of small beam: W, label: W5X19
big beam input: tw=12, tf=21.5, b=300, d=340
big beam retrieved: tw=15.5, tf=25.1, b=310.0, d=328.0
error: tw=0.2916666666666667, tf=0.16744186046511633, b=0.03333333333333333, d=0.03529411764705882
type of big beam: W, label: W12X106


In [27]:
import os
import pandas as pd
import numpy as np


def huber_loss_local(residual, delta):
    if abs(residual) <= delta:
        return 0.5 * residual ** 2
    else:
        return delta * (abs(residual) - 0.5 * delta)


def standalone_lookup(tf, tw, b, d):
    path = '/data/beams/aisc-shapes-database-v15.0.csv'
    path = os.getcwd() + path

    path = '/Users/fnoic/PycharmProjects/reconstruct/data/beams/aisc-shapes-database-v15.0.csv'

    with open(path, 'r', newline='\n') as f:
        beams = pd.read_csv(f, header=0, sep=';')
        # retrieve name of first column
        uno = beams.columns[0]
        beams_frame = beams[[uno, 'AISC_Manual_Label', 'tw.1', 'tf.1', 'bf.1', 'd.1']]
        # rename columns
        beams_frame.columns = ['type', 'label', 'tw', 'tf', 'bf', 'd']
        # remove all "â€“", replace with nan
        beams_frame = beams_frame.replace('â€“', np.nan, regex=True)
        # replace all , with . for tw tf bf and d
        beams_frame = beams_frame.replace(',', '.', regex=True)
        # drop all rows with –
        beams_frame = beams_frame.replace('–', np.nan, regex=True)
        # convert to numeric in column tw
        beams_frame[['tw', 'tf', 'bf', 'd']] = beams_frame[['tw', 'tf', 'bf', 'd']].apply(pd.to_numeric)
        # beams_frame = beams_frame.apply(pd.to_numeric)
        # convert to string in column label
        beams_frame['label'] = beams_frame['label'].astype(str)
        # drop all rows where label starts with "WT"
        beams_frame = beams_frame[~beams_frame['label'].str.startswith('WT')]

    # divide the selected columns by 1000 to convert to mm
    beams_frame[['tw', 'tf', 'bf', 'd']] = beams_frame[['tw', 'tf', 'bf', 'd']] # / 1e3

    # find the closest beam
    tw_fit = tw
    tf_fit = tf
    bf_fit = b
    d_fit = d

    # find best fit row in beams_frame with Huber Loss
    delta = 1.0  # TODO: include in config
    beams_frame['HuberLoss'] = (
            beams_frame.apply(lambda row: huber_loss_local(row['tw'] - tw_fit, delta), axis=1) +
            beams_frame.apply(lambda row: huber_loss_local(row['tf'] - tf_fit, delta), axis=1) +
            beams_frame.apply(lambda row: huber_loss_local(row['bf'] - bf_fit, delta), axis=1) +
            beams_frame.apply(lambda row: huber_loss_local(row['d'] - d_fit, delta), axis=1)
    )
    beams_frame = beams_frame.sort_values(by='HuberLoss', ascending=True)


    tw_lookup = beams_frame['tw'].iloc[0]
    tf_lookup = beams_frame['tf'].iloc[0]
    bf_lookup = beams_frame['bf'].iloc[0]
    d_lookup = beams_frame['d'].iloc[0]
    type_lookup = beams_frame['type'].iloc[0]
    label_lookup = beams_frame['label'].iloc[0]
    

    return {
        'tw': tw_lookup,
        'tf': tf_lookup,
        'bf': bf_lookup,
        'd': d_lookup,
        'type': type_lookup,
        'label': label_lookup
    }